In [13]:
import boto3 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import recall_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

#defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'data445-ashlyn-bucket'
bucket = s3.Bucket(bucket_name)

#defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

#reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [14]:
#input and target variables
X = heart[['age', 'totChol','sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

#splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = .2, stratify = Y)

In [15]:
#0-1 scale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [16]:
#adaboost
md1 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500, learning_rate = 0.01).fit(X_train, Y_train)

#predicting on test
pred1 = md1.predict_proba(X_test)[:, 1]
pred1_label = np.where(pred1 < 0.15, 0, 1)

#computing accurary and recall score
print('The accuracy is', accuracy_score(Y_test, pred1_label))
print('The recall is', recall_score(Y_test, pred1_label))

The accuracy is 0.17349726775956284
The recall is 0.9553571428571429


In [17]:
#gradient boost
md2 = GradientBoostingClassifier(n_estimators = 500, max_depth = 3, learning_rate = 0.01).fit(X_train, Y_train)

#predicting on test
pred2 = md2.predict_proba(X_test)[:, 1]
pred2_label = np.where(pred2 < 0.15, 0, 1)

#computing accurary and recall score
print('The accuracy is', accuracy_score(Y_test, pred2_label))
print('The recall is', recall_score(Y_test, pred2_label))

The accuracy is 0.5532786885245902
The recall is 0.8035714285714286


In [18]:
#support vector model
md3 = SVC(kernel = 'rbf', probability = True).fit(X_train, Y_train)

#predicting on test
pred3 = md3.predict_proba(X_test)[:, 1]
pred3_label = np.where(pred3 < 0.15, 0, 1)

#computing accurary and recall score
print('The accuracy is', accuracy_score(Y_test, pred3_label))
print('The recall is', recall_score(Y_test, pred3_label))

The accuracy is 0.6188524590163934
The recall is 0.5892857142857143


In [19]:
#ensemble of likelihood
X_rf = pd.concat([pd.DataFrame(pred1), pd.DataFrame(pred2), pd.DataFrame(pred3), Y_test.reset_index(drop = True)], axis = 1)

#defining input and target variables
X_rf1 = pd.concat([pd.DataFrame(pred1), pd.DataFrame(pred2), pd.DataFrame(pred3)], axis = 1)

#random forest model
RF_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_rf1, Y_test)

#extracting ensemble likelihood
RF_pred = RF_md.predict_proba(X_rf1)[:, 1]

#final results
final_results = pd.concat([pd.DataFrame(pred1), pd.DataFrame(pred2), pd.DataFrame(pred3), pd.DataFrame(RF_pred), Y_test.reset_index(drop = True)], axis = 1)
final_results.head()             

,0,0,0,0,TenYearCHD
0,0.425577,0.227314,0.193821,0.337794,0
1,0.319988,0.097180,0.135346,0.086229,0
2,0.387159,0.328974,0.173757,0.215051,0
3,0.426162,0.139034,0.150952,0.125367,0
4,0.423528,0.311913,0.151024,0.181041,0
